In [17]:
import numpy as np
import pandas
from helper_scripts import random_rm_trace_generator
import importlib
from matplotlib import pyplot as plt
importlib.reload(random_rm_trace_generator)

<module 'helper_scripts.random_rm_trace_generator' from '/cr/work/kastner/private/umap/rm_classifier/tutorials/notebooks/helper_scripts/random_rm_trace_generator.py'>

In [ ]:
spectra, time_traces, clusters, _,_=random_rm_trace_generator.generate_random_rm_traces(400, n_clusters=10, outlier_fraction=0.05)
for i in range(400):
    if clusters[i] == -1:
        plt.plot(spectra[i], label=f'Trace {i+1}')